# Epidemic tagging
Ya habiendo procesado los simple_fromat files para saber cuales son las top 10 antennas utilizadas por c/user ahora falta agregar la inforamcion epidemica/de exposicion (EPIDEMIC/EXPOSED) p/c/user.

* Se define epidemico si su Antenna_ID_0 es una antenna taggeada como epidemica (la 0 es la mas frecuentemente utilizada)
* Se define exposed si utlizo alguna vez una antenna taggeada como epidemica (obs. hay mas usuarios expuestos que epidemicos)

In [47]:
%autosave 180
import pandas as pd; import numpy as np; import os;import random;
pd.set_option('display.max_rows', 300)
#from matplotlib import pyplot as plt
#%matplotlib inline
import time
import os
np.random.seed(2016)

Autosaving every 180 seconds


### el dataset de input vendria con este header

In [48]:
#ejemplo  "/grandata/simple_format/01-02012014.txt.gz"
#seteamos el lugar de trabajo
input_file = "/home/juan/mobility-study/play_ants/sf_crude.txt.gz"

user_table = "/home/juan/mobility-study/play_ants/sf_user_table"

!du -ha {input_file}

!zcat $input_file | head

173M	/home/juan/mobility-study/play_ants/sf_crude.txt.gz
USER|ANTENNA_ID_0|COUNT_0|ANTENNA_ID_1|COUNT_1|ANTENNA_ID_2|COUNT_2|ANTENNA_ID_3|COUNT_3|ANTENNA_ID_4|COUNT_4|ANTENNA_ID_5|COUNT_5|ANTENNA_ID_6|COUNT_6|ANTENNA_ID_7|COUNT_7|ANTENNA_ID_8|COUNT_8|ANTENNA_ID_9|COUNT_9|MOBILITY_DIAMETER|EPIDEMIC|EXPOSED
FFFF78CB080636632B2D1DE7A7BFAB03|3415|21|2969|5|1320|3|2471|3|1115|2|-1|65535|-1|65535|-1|65535|-1|65535|-1|65535|1063|0|1
FFFF51FDDD55A2E24D074DE30C4798E9|1207|15|1171|14|1820|9|583|8|2026|5|1066|4|1831|4|1625|2|401|1|825|1|520|0|1
FFFE1EDDA25369CBC467536FA6A787FD|1847|22|1753|14|1422|11|1028|10|765|9|3072|9|862|8|2463|7|1416|6|871|5|990|0|1
FFFD0A912A959CDFCFFAB2F93BFBA435|1407|18|1702|8|1613|6|1704|5|1394|3|1398|3|818|2|958|2|726|1|848|1|491|1|0
FFFCB235D6D917DDF350F5B11CCB0EE8|17|74|174|33|1123|4|1178|4|173|3|890|2|503|1|1039|1|1228|1|1327|1|1613|0|1
FFFC9EC4F662C44D3006BA69C9F0DE85|876|19|1737|6|1744|5|326|1|378|1|503|1|1118|1|2216|1|2477|1|2962|1|3004|0|1
FFFB2F18F4CCF253C06B581

In [79]:
#para enriquecer el dataset de CDRs con info acerca de la zona epidemica etc.
antennas = pd.read_csv('/home/juan/mobility-study/play_ants/antennas_mexico.csv',
                      sep = "|",
                      header=0,
                      index_col=0)
antennas.index.rename("ANTENNA_ID_0",inplace=True)

In [74]:
#levanto la tabla de users hash_map.
user_epidemic = pd.read_csv(input_file,
                    sep="|",
                    index_col=0,
                    header=0,
                    usecols = ["USER","ANTENNA_ID_0","ANTENNA_ID_1","ANTENNA_ID_2","ANTENNA_ID_3","ANTENNA_ID_4",
                                "ANTENNA_ID_5","ANTENNA_ID_6","ANTENNA_ID_7",
                        "ANTENNA_ID_8","ANTENNA_ID_9"]
                    
                )

In [80]:
antennas.head()

,LATITUDE,LONGITUDE,STATE,EPIDEMIC
ANTENNA_ID_0,,,,
2797,14.671,-92.372,Chiapas,0
3060,14.682,-92.155,Chiapas,0
3344,14.721,-92.424,Chiapas,0
3487,14.778,-92.179,Chiapas,0
1924,14.786,-92.366,Chiapas,0


In [82]:
user_epidemic.columns[0] == antennas.index.name

True

In [62]:
antennas.dtypes

ANTENNA_ID_0      int64
LATITUDE        float64
LONGITUDE       float64
STATE            object
EPIDEMIC          int64
dtype: object

# Mega bug 02/05
en este merge donde el left merge me agranda (!!) la tabla original y ademas vemos que aparecen nans pues hay keys que no matchean i.e. hay users que vivirian en antennas que no existen en nuestra db de antennas.

Sin embargo cuando sampleamos a estas antennas "fantasmas", resulta que no podemos encontrar alguna que falte. i.e. siempre estan todas

In [63]:
user_ants = np.unique(user_epidemic.ANTENNA_ID_0.values)
db_ants = np.unique(antennas.ANTENNA_ID_0.values)
sanity = np.in1d(user_ants , db_ants)
sanity2 = np.in1d(db_ants ,user_ants )
len(user_ants), sanity.sum(), len(db_ants), sanity2.sum()

(4185, 4185, 4429, 4185)

## Result
No habiamos cambiado los files viejos de surrogate_antennas

In [83]:
antennas.head()

,LATITUDE,LONGITUDE,STATE,EPIDEMIC
ANTENNA_ID_0,,,,
2797,14.671,-92.372,Chiapas,0
3060,14.682,-92.155,Chiapas,0
3344,14.721,-92.424,Chiapas,0
3487,14.778,-92.179,Chiapas,0
1924,14.786,-92.366,Chiapas,0


In [69]:
!zcat /home/juan/mobility-study/surrogated_antennas.txt.gz | head

ID LATITUDE LONGITUDE
2797 14671 -92372
3060 14682 -92155
3344 14721 -92424
3487 14778 -92179
1924 14786 -92366
1001 14870 -92285
2914 14882 -92265
2094 14888 -92283
2055 14888 -92250

gzip: stdout: Broken pipe


In [84]:
print(user_epidemic.shape)
test = user_epidemic.join(antennas['EPIDEMIC'], on = user_epidemic.columns[0],
           how='left', sort=False,rsuffix="_r")

print(test.shape)

(3667451, 10)
(3667451, 11)


In [88]:
for col in user_epidemic.columns:
    antennas.index.name = col
    user_epidemic = user_epidemic.join(antennas['EPIDEMIC'],on = col,how = 'left',rsuffix= col.replace("ANTENNA_ID_","")).fillna(0)
    print(col)
    print(user_epidemic.shape)
    #user_epidemic[col.replace("ANTENNA_ID_","EPIDEMIC_")] = (user_epidemic[col]>0).astype(int)

ANTENNA_ID_0
(3667451, 11)
ANTENNA_ID_1
(3667451, 12)
ANTENNA_ID_2
(3667451, 13)
ANTENNA_ID_3
(3667451, 14)
ANTENNA_ID_4
(3667451, 15)
ANTENNA_ID_5
(3667451, 16)
ANTENNA_ID_6
(3667451, 17)
ANTENNA_ID_7
(3667451, 18)
ANTENNA_ID_8
(3667451, 19)
ANTENNA_ID_9
(3667451, 20)


In [86]:
print(len(np.unique(user_epidemic.index.values)))
print(user_epidemic.shape)

3667451
(3667451, 10)


In [90]:
#agrego la definicion de expuesto que es fijarme si uso alguna vez alguna antenna epidemica
user_epidemic['EXPOSED'] = (user_epidemic[[col for col in user_epidemic.columns if "EPIDEMIC" in col]].sum(axis=1)>0).astype(int)

drop_cols = ["EPIDEMIC"+str(col) for col in range(1,10)]
user_epidemic.drop(drop_cols,axis=1,inplace=True)
#necesito este cambio de nombre para despues
user_epidemic.index.name = 'LineKeyOrigin'
#por temas de memoria dropeo las columnas que no voy a usar
user_epidemic = user_epidemic[['ANTENNA_ID_0',"EPIDEMIC","EXPOSED"]]

In [91]:
user_epidemic.head()

,ANTENNA_ID_0,EPIDEMIC,EXPOSED
LineKeyOrigin,,,
FFFF78CB080636632B2D1DE7A7BFAB03,3415,0,0
FFFF51FDDD55A2E24D074DE30C4798E9,1207,0,1
FFFE1EDDA25369CBC467536FA6A787FD,1847,0,1
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,0,1
FFFCB235D6D917DDF350F5B11CCB0EE8,17,0,1


In [92]:
table = pd.read_csv( "/home/juan/mobility-study/play_ants/sl_crude.txt.gz",
                              header =0,
                              index_col=None,
                              engine="c",
                              sep="|",
                               #dtype = {'CallsWeekDaylight':np.uint16,'CallsWeekNight':np.uint16,'CallsWeekend':np.uint16,
                                #          'TimeWeekDaylight':np.uint16,'TimeWeekNight':np.uint16,'TimeWeekend':np.uint16}                          
                                )

cols = [col for col in table.columns if not('Line' in col) ]
#levanto todo en float, lo paso a int (excepto los hashes)
table[cols] = table[cols].astype(int)

#agrego la info de epidemicidad de los targets.
table = table.join(user_epidemic[['EPIDEMIC','ANTENNA_ID_0']], on='LineKeyOrigin',how='left',rsuffix='_Origin')
table = table.join(user_epidemic[['EPIDEMIC','ANTENNA_ID_0']], on='LineKeyTarget',how='left',rsuffix='_Target')
table.rename(columns={'EPIDEMIC':"EPIDEMIC_Origin", 'ANTENNA_ID_0':"ANTENNA_ID_0_Origin"},inplace=True)
#del user_epidemic

In [93]:
#este helper es para contar "lexicograficamente" valores unicos en un array de 2 columnas
def unique(array):
    order = np.lexsort(array.T)
    array = array[order]
    diff = np.diff(array, axis=0)
    ui = np.ones(len(array), 'bool')
    ui[1:] = (diff != 0).any(axis=1) 
    return array[ui]

prueba = np.array([[1,2],[1,3],[1,3],[1,5],[2,1],[2,1]]) 
unique(prueba)

array([[2, 1],
       [1, 2],
       [1, 3],
       [1, 5]])

In [94]:
%%time
#relevant columns
rel_cols = ['LineKeyOrigin'] + [col for col in table.columns if ('Calls' in col) or ('Time' in col) ] 
#toda la info de llamados
all_info = table[rel_cols].groupby('LineKeyOrigin').sum()
#solo los llamados "epi"
epi_info = table[table['EPIDEMIC_Target']==1][rel_cols].groupby('LineKeyOrigin').sum()
epi_info.columns = [col+"_EPI" for col in epi_info.columns]

result = pd.concat([all_info, epi_info], axis=1)
del all_info, epi_info
#ojo que este concat introduce nulls donde no hay index compartido entre los dataframes pues el join es 'outer'

#reagrego la columna linekeyorigin
result.index.name=rel_cols[0]
#result.reset_index(level =0, inplace=True)
# sin embargo no esperamos que haya nulls en las columnas relevantes
print(result[rel_cols[1:]].isnull().sum().sum())
#entonces rellenamos los valores faltantes con 0..
result = result.fillna(0)

0
CPU times: user 7.21 s, sys: 1.41 s, total: 8.62 s
Wall time: 8.67 s


## Direccion de llamado
SumLinks tiene en cuenta la direccion del llamado pero en eseta celda vamos a agregar esta informacion, luego debemos rotar la informacion de la dataframe de sum_links i.e. swapear origin con target y toda esa info relevante para correr el mismo algoritmo.

In [95]:
table.head(2)

,LineKeyOrigin,LineKeyTarget,CallsWeekDaylight,CallsWeekNight,CallsWeekend,TimeWeekDaylight,TimeWeekNight,TimeWeekend,EPIDEMIC_Origin,ANTENNA_ID_0_Origin,EPIDEMIC_Target,ANTENNA_ID_0_Target
0,00000837B868F926F67BD440A0845C1D,5350FBED91E7DA465D7D436ADD1C48F3,0,0,2,0,0,45,0,650,0,650
1,00000837B868F926F67BD440A0845C1D,C88DD6FDDF73AF80517291EEA7339B32,1,2,1,47,74,64,0,650,0,650


In [96]:
#swapeo los nombres de las columnas, dropeo epidemic y lo vuelvo a agregar para los nuevos users swappeados
table=table.rename(columns = {'LineKeyOrigin':'LineKeyTarget',
                        'LineKeyTarget':'LineKeyOrigin',
                             'EPIDEMIC_Origin':'EPIDEMIC_Target',
                              'EPIDEMIC_Target':'EPIDEMIC_Origin'})

In [97]:
table.head(2)

,LineKeyTarget,LineKeyOrigin,CallsWeekDaylight,CallsWeekNight,CallsWeekend,TimeWeekDaylight,TimeWeekNight,TimeWeekend,EPIDEMIC_Target,ANTENNA_ID_0_Origin,EPIDEMIC_Origin,ANTENNA_ID_0_Target
0,00000837B868F926F67BD440A0845C1D,5350FBED91E7DA465D7D436ADD1C48F3,0,0,2,0,0,45,0,650,0,650
1,00000837B868F926F67BD440A0845C1D,C88DD6FDDF73AF80517291EEA7339B32,1,2,1,47,74,64,0,650,0,650


In [98]:
rel_cols = ['LineKeyOrigin'] + [col for col in table.columns if ('Calls' in col) or ('Time' in col) ] 
all_info = table[rel_cols].groupby('LineKeyOrigin').sum()
epi_info = table[table['EPIDEMIC_Target']==1][rel_cols].groupby('LineKeyOrigin').sum()
epi_info.columns = [col+"_EPI" for col in epi_info.columns]

user_table = pd.concat([all_info, epi_info], axis=1)
#ojo que este concat introduce nulls donde no hay index compartido entre los dataframes pues el join es 'outer'
del all_info, epi_info
#reagrego la columna linekeyorigin
user_table.index.name=rel_cols[0]
#user_table.reset_index(level =0, inplace=True)
# sin embargo no esperamos que haya nulls en las columnas relevantes
print(user_table[rel_cols[1:]].isnull().sum().sum())
#entonces rellenamos los valores faltantes con 0..
user_table = user_table.fillna(0)

0


In [99]:
#junto las dos tablas sumando la info
user_table = result.add(user_table,fill_value=0)
user_table = user_table.astype(int)
del result

In [100]:
#checking that all "_EPI" should be smaller or equal then their non_EPI column
for col in rel_cols:
    if col == 'LineKeyOrigin':
        continue
    epi_col = col+"_EPI"
    print("col {0} is <= than col {1}: {2}".format(epi_col,col,
                                (user_table[epi_col] <= user_table[col]).all()  ))

table_hashes = set(table.LineKeyOrigin.values) | set(table.LineKeyTarget.values)

print(np.in1d(table.LineKeyOrigin.values, user_table.index.values ).all())
print( len(table_hashes),len(user_table) )

col CallsWeekDaylight_EPI is <= than col CallsWeekDaylight: True
col CallsWeekNight_EPI is <= than col CallsWeekNight: True
col CallsWeekend_EPI is <= than col CallsWeekend: True
col TimeWeekDaylight_EPI is <= than col TimeWeekDaylight: True
col TimeWeekNight_EPI is <= than col TimeWeekNight: True
col TimeWeekend_EPI is <= than col TimeWeekend: True
True
(2449242, 2449242)


In [101]:
print(table.shape)

(5015519, 12)


In [102]:
# me genero la tabla con los pares (i,j) y concateno con la tabla pero swappeando en (j,i) para lograr hacer los counts apropiados

user_pair = table[['LineKeyOrigin','LineKeyTarget']].copy()
#no necesitamos mas esta info
del table

In [103]:
print(user_pair.shape)

(5015519, 2)


## OJO
el proximo join puede reventar facilmente la memoria del server

tal vez sea mejor guardar el file al disco para restartear el kernel, liberar memoria y volver a arrancar

In [105]:
users1 = np.unique(user_pair.LineKeyOrigin.values)
users2 = np.unique(user_pair.LineKeyTarget.values)
sanity1 = np.in1d(users1,user_epidemic.index.values)
sanity2 = np.in1d(users2,user_epidemic.index.values)
len(users1), len(users2), sanity1.sum(), sanity2.sum()

(2101406, 1927239, 2101406, 1927239)

In [108]:
#agrego la informacion epidemica y de exposicion
user_pair = user_pair.join(user_epidemic[['EPIDEMIC','EXPOSED']].astype(int),on = 'LineKeyTarget',
                           how = 'left',rsuffix='_Target')

In [109]:
user_epidemic.head()

,ANTENNA_ID_0,EPIDEMIC,EXPOSED
LineKeyOrigin,,,
FFFF78CB080636632B2D1DE7A7BFAB03,3415,0,0
FFFF51FDDD55A2E24D074DE30C4798E9,1207,0,1
FFFE1EDDA25369CBC467536FA6A787FD,1847,0,1
FFFD0A912A959CDFCFFAB2F93BFBA435,1407,0,1
FFFCB235D6D917DDF350F5B11CCB0EE8,17,0,1


In [110]:
print(len(np.unique(user_epidemic.index.values)))
print(user_epidemic.shape)

3667451
(3667451, 3)


In [111]:
user_pair.head()

,LineKeyOrigin,LineKeyTarget,EPIDEMIC,EXPOSED
0,5350FBED91E7DA465D7D436ADD1C48F3,00000837B868F926F67BD440A0845C1D,0,1
1,C88DD6FDDF73AF80517291EEA7339B32,00000837B868F926F67BD440A0845C1D,0,1
2,11509D6701C5280D0D0B2271246424B0,000017368A8964F8C71A9F3A1EAF6160,1,1
3,651D2B4E897DB722F28EDB856F4909FE,000017368A8964F8C71A9F3A1EAF6160,1,1
4,FB36428E265F633EC2227140BDDCCB4D,000017368A8964F8C71A9F3A1EAF6160,1,1


In [112]:
print(user_pair.shape)

(5015519, 4)


In [113]:
user_pair2 = user_pair[['LineKeyOrigin','LineKeyTarget']].copy()
#reordeno las columnas
user_pair2.columns = ['LineKeyOrigin','LineKeyTarget']
user_pair = pd.concat([user_pair,user_pair2])
del user_pair2

In [114]:
%%time

# UNIQUE # of users count

#primero agrupo por el par (i,j) para no tener dos veces el par y agrupo 
#(el agregador es "bobo" no voy a estar usando la info de esta agrupacion)
total_users = user_pair.groupby(['LineKeyOrigin','LineKeyTarget']).count()
total_users = total_users.reset_index()
total_users.rename(columns={'EPIDEMIC':'TOTAL_USERS'},inplace=True)
# ahora conteo los usuarios unicos
total_users = total_users.groupby('LineKeyOrigin')['TOTAL_USERS'].count()
total_users = total_users.to_frame()
total_users = total_users.reset_index()

# EPIDEMIC & EXPOSED count
#primero agrupo por el par (i,j) para no tener dos veces el par y agrupo 
#(el agregador es "bobo" no voy a estar usando la info de esta agrupacion)

epidemic_count = user_pair[user_pair['EPIDEMIC']==1]\
                .groupby(['LineKeyOrigin','LineKeyTarget'])['EPIDEMIC'].count()
epidemic_count = epidemic_count.reset_index()
epidemic_count.rename(columns={'EPIDEMIC':'EPI_USERS'},inplace=True)

exposed_count = user_pair[user_pair['EXPOSED']==1]\
                .groupby(['LineKeyOrigin','LineKeyTarget'])['EXPOSED'].count()
exposed_count = exposed_count.reset_index()
exposed_count.rename(columns={'EXPOSED':'EXP_USERS'},inplace=True)

#reseteo el indice
#epidemic_count = epidemic_count.reset_index()
#exposed_count = exposed_count.reset_index()

#ahora si agrupo y cuento c/ par unico (Origin,Target)
epidemic_count = epidemic_count.groupby(['LineKeyOrigin'])['EPI_USERS'].count()
exposed_count = exposed_count.groupby(['LineKeyOrigin'])['EXP_USERS'].count()

epidemic_count = epidemic_count.to_frame()
exposed_count = exposed_count.to_frame()
del user_pair

CPU times: user 56.4 s, sys: 2.73 s, total: 59.1 s
Wall time: 59.5 s


In [115]:
#agrego la informacion epidemica y de exposicion en total_users y relleno los NaNs con 0s i.e. ausencia de conteos
total_users = total_users.join(epidemic_count['EPI_USERS'],on = 'LineKeyOrigin', how = 'left').fillna(0)
total_users = total_users.join(exposed_count['EXP_USERS'],on = 'LineKeyOrigin', how = 'left').fillna(0)

del exposed_count, epidemic_count

total_users.set_index('LineKeyOrigin',inplace=True)

print("total algo time for {n} users is ".format(n=(len(total_users))))

total algo time for 2101406 users is 


In [116]:
#check that all "_EPI" should be smaller or equal then their non_EPI column
tot_cols =['TOTAL_USERS','EPI_USERS','EXP_USERS']
print("col {0} is >= than col {1}: {2}".format(tot_cols[0],tot_cols[1],
                                (total_users[tot_cols[0]] >= total_users[tot_cols[1]]).all()  ))
print("col {0} is >= than col {1}: {2}".format(tot_cols[0],tot_cols[2],
                                (total_users[tot_cols[0]] >= total_users[tot_cols[2]]).all()  ))
print("col {0} is >= than col {1}: {2}".format(tot_cols[2],tot_cols[1],
                                (total_users[tot_cols[2]] >= total_users[tot_cols[1]]).all()  ))

col TOTAL_USERS is >= than col EPI_USERS: True
col TOTAL_USERS is >= than col EXP_USERS: True
col EXP_USERS is >= than col EPI_USERS: True


In [117]:
#add info to user_table table
user_table = pd.merge(user_table,total_users,how='inner',right_index=True,left_index=True).astype(int)
del total_users

In [118]:
#aggregating total_users, total_epi_users and total_exposed_users info

user_table = user_table.join(user_epidemic['ANTENNA_ID_0'],how='left',rsuffix='_Origin')
calls_cols = [col for col in user_table.columns if "Calls" in col]
time_cols =  [col for col in user_table.columns if "Time" in col]

epi_calls_cols = [col for col in user_table.columns if ("Calls" in col and "EPI" in col)]
epi_time_cols =  [col for col in user_table.columns if ("Time" in col and "EPI" in col)]

user_table['TOTAL_Calls'] = user_table[calls_cols].sum(axis =1).values
user_table['TOTAL_Time'] = user_table[time_cols].sum(axis =1).values

user_table['TOTAL_EPI_Calls'] = user_table[epi_calls_cols].sum(axis = 1).values
user_table['TOTAL_EPI_Time'] = user_table[epi_time_cols].sum(axis = 1).values

calls_cols = [col for col in user_table.columns if "Calls" in col]
time_cols =  [col for col in user_table.columns if "Time" in col]

antenna2vuln = user_table.groupby('ANTENNA_ID_0')[calls_cols + time_cols].sum()

# 1EPI_USER vendria a ser como if USER Interacted with at least 1 epidemic user in that month 
user_table['1EPI_USER'] = (user_table['EPI_USERS']>=1).astype(int).values
user_table['1EXP_USER'] = (user_table['EXP_USERS']>=1).astype(int).values

# como el groupby es sorted by default, no tengo el problema del desorden aca.
antenna2vuln['EXP_USERS'] = user_table.groupby('ANTENNA_ID_0')['1EXP_USER'].sum()
antenna2vuln['EPI_USERS'] = user_table.groupby('ANTENNA_ID_0')['1EPI_USER'].sum()
antenna2vuln['TOTAL_USERS'] = user_table.groupby('ANTENNA_ID_0')['TOTAL_USERS'].count()

In [119]:
#reordenamos las columnas como querriamos
old_cols = """TOTAL_USERS,EPI_USERS,EXP_USERS,
CallsWeekDaylight,CallsWeekNight,CallsWeekend,TimeWeekDaylight,TimeWeekNight,TimeWeekend,
CallsWeekDaylight_EPI,CallsWeekNight_EPI,CallsWeekend_EPI,TOTAL_Calls,TOTAL_EPI_Calls,TimeWeekDaylight_EPI,
TimeWeekNight_EPI,TimeWeekend_EPI,TOTAL_Time,TOTAL_EPI_Time"""
new_cols =old_cols.replace("\n","").replace(" ","").split(",")

# chequeo consistencia del reorden
print(antenna2vuln[new_cols].shape, antenna2vuln.shape)

antenna2vuln = antenna2vuln[new_cols]

((4178, 19), (4178, 19))


In [120]:
ant_file = "/home/juan/mobility-study/play_ants/ant_vuln_info.txt"
antenna2vuln.to_csv(ant_file, 
             index = True, 
             header = True,
             float_format='%.0f',
             sep="|")